<a href="https://colab.research.google.com/github/ameerjasser/data-science-toolkit/blob/main/Final_Project_First_Semester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Science in Finance & FinTech**

**Banks run into losses when customers don't pay their loans on time. Because of this, every year, banks have losses in crores, and this also impacts the country's economic growth to a large extent. In this project, we look at various attributes such as funded amount, location, loan, balance, etc., to predict whether a person will be a loan defaulter.**

In [43]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Science/Project/Dataset.csv') #load the csv dataset into a 'dataframe' df

In [45]:
df.head()              # see the first 5 columns

,ID,Loan Amount,Funded Amount,Funded Amount Investor,Term,Batch Enrolled,Interest Rate,Grade,Sub Grade,Employment Duration,...,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Application Type,Last week Pay,Accounts Delinquent,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,Loan Status
0,56492997,17120,10365,16025.082690,59,BAT2575549,12.163926,A,D1,RENT,...,8.425776,0.731797,0,INDIVIDUAL,135,0,24,475442,4364,NaN
1,22540813,7133,11650,12615.795600,59,BAT2833642,6.564296,B,E3,MORTGAGE,...,6.157008,0.992918,0,INDIVIDUAL,56,0,1,72412,2573,NaN
2,9862181,25291,25825,11621.280830,59,BAT1761981,14.729900,A,C3,MORTGAGE,...,5.705077,0.281580,0,INDIVIDUAL,3,0,26,284825,19676,NaN
3,10097822,30781,9664,15375.823510,59,BAT5341619,10.523767,A,A2,RENT,...,2.469688,0.959162,0,INDIVIDUAL,21,0,32,40842,7226,NaN
4,47771809,8878,9419,7176.647582,58,BAT4694572,9.997013,C,B3,OWN,...,2.127835,0.402315,0,INDIVIDUAL,104,0,33,90825,26145,NaN


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28913 entries, 0 to 28912
Data columns (total 35 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            28913 non-null  int64  
 1   Loan Amount                   28913 non-null  int64  
 2   Funded Amount                 28913 non-null  int64  
 3   Funded Amount Investor        28913 non-null  float64
 4   Term                          28913 non-null  int64  
 5   Batch Enrolled                28913 non-null  object 
 6   Interest Rate                 28913 non-null  float64
 7   Grade                         28913 non-null  object 
 8   Sub Grade                     28913 non-null  object 
 9   Employment Duration           28913 non-null  object 
 10  Home Ownership                28913 non-null  float64
 11  Verification Status           28913 non-null  object 
 12  Payment Plan                  28913 non-null  object 
 13  L

In [47]:
# --- FIX THE MISALIGNED DATA ---

# Check unique values for all 'object' or suspected categorical columns
for col in df.columns:
    unique_count = df[col].nunique()
    print(f"Column: {col} | Unique Values: {unique_count} | Dtype: {df[col].dtype}")

    # If a column that should be a category (like 'Grade' or 'Status')
    # has thousands of unique float values, it is mislabeled.
    if unique_count > 100 and df[col].dtype in ['float64', 'int64']:
         print(f"  --> ALERT: {col} might be mislabeled (too many unique numbers)")
    print("\n\n")


Column: ID | Unique Values: 28913 | Dtype: int64
  --> ALERT: ID might be mislabeled (too many unique numbers)



Column: Loan Amount | Unique Values: 15664 | Dtype: int64
  --> ALERT: Loan Amount might be mislabeled (too many unique numbers)



Column: Funded Amount | Unique Values: 14552 | Dtype: int64
  --> ALERT: Funded Amount might be mislabeled (too many unique numbers)



Column: Funded Amount Investor | Unique Values: 22419 | Dtype: float64
  --> ALERT: Funded Amount Investor might be mislabeled (too many unique numbers)



Column: Term | Unique Values: 4 | Dtype: int64



Column: Batch Enrolled | Unique Values: 41 | Dtype: object



Column: Interest Rate | Unique Values: 21675 | Dtype: float64
  --> ALERT: Interest Rate might be mislabeled (too many unique numbers)



Column: Grade | Unique Values: 7 | Dtype: object



Column: Sub Grade | Unique Values: 35 | Dtype: object



Column: Employment Duration | Unique Values: 3 | Dtype: object



Column: Home Ownership | Unique Value

In [48]:
# --- FIX THE MISALIGNED DATA ---

# 1. Temporarily store the actual Home Ownership status (RENT/MORTGAGE/OWN)
#    from the 'Employment Duration' column.
temp_home_ownership_status = df['Employment Duration']

# 2. Move the Annual Income values from 'Home Ownership' to a new, correctly named column.
df = df.rename(columns={'Home Ownership': 'Annual_Income'})

# 3. Assign the actual Home Ownership status to its correctly named column.
#    Since 'Employment Duration' might be needed later, we can rename that column as well.
df = df.rename(columns={'Employment Duration': 'Home_Ownership'})

In [49]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28913 entries, 0 to 28912
Data columns (total 35 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            28913 non-null  int64  
 1   Loan Amount                   28913 non-null  int64  
 2   Funded Amount                 28913 non-null  int64  
 3   Funded Amount Investor        28913 non-null  float64
 4   Term                          28913 non-null  int64  
 5   Batch Enrolled                28913 non-null  object 
 6   Interest Rate                 28913 non-null  float64
 7   Grade                         28913 non-null  object 
 8   Sub Grade                     28913 non-null  object 
 9   Home_Ownership                28913 non-null  object 
 10  Annual_Income                 28913 non-null  float64
 11  Verification Status           28913 non-null  object 
 12  Payment Plan                  28913 non-null  object 
 13  L

When column names are lowercase and have no spaces, you can access them as attributes rather than using brackets and quotes.

Standardizes and Reduces Errors

Follows Python Best Practices (PEP 8)

In [50]:
# 1. Basic cleaning: lowercase and replace spaces/hyphens with underscores
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '')

# 2. Specific renaming for clarity/typos
df = df.rename(columns={
    'id': 'loan_id',
    'inquires__six_months': 'inquiries_6m',
    'delinquency__two_years': 'delinquency_2y',
    'debit_to_income': 'dti',
    'collection_12_months_medical': 'med_collections_12m'
})

print(df.columns)


Index(['loan_id', 'loan_amount', 'funded_amount', 'funded_amount_investor',
       'term', 'batch_enrolled', 'interest_rate', 'grade', 'sub_grade',
       'home_ownership', 'annual_income', 'verification_status',
       'payment_plan', 'loan_title', 'dti', 'delinquency_2y', 'inquiries_6m',
       'open_account', 'public_record', 'revolving_balance',
       'revolving_utilities', 'total_accounts', 'initial_list_status',
       'total_received_interest', 'total_received_late_fee', 'recoveries',
       'collection_recovery_fee', 'med_collections_12m', 'application_type',
       'last_week_pay', 'accounts_delinquent', 'total_collection_amount',
       'total_current_balance', 'total_revolving_credit_limit', 'loan_status'],
      dtype='object')


In [51]:
# Write a function that returns the each column name with the number of missing values.

print('--Columns and the number of missing values--')

for column_name in df.columns:
  n_missing = df[column_name].isna().sum()
  print(f'{column_name} : {n_missing}')

--Columns and the number of missing values--
loan_id : 0
loan_amount : 0
funded_amount : 0
funded_amount_investor : 0
term : 0
batch_enrolled : 0
interest_rate : 0
grade : 0
sub_grade : 0
home_ownership : 0
annual_income : 0
verification_status : 0
payment_plan : 0
loan_title : 0
dti : 0
delinquency_2y : 0
inquiries_6m : 0
open_account : 0
public_record : 0
revolving_balance : 0
revolving_utilities : 0
total_accounts : 0
initial_list_status : 0
total_received_interest : 0
total_received_late_fee : 0
recoveries : 0
collection_recovery_fee : 0
med_collections_12m : 0
application_type : 0
last_week_pay : 0
accounts_delinquent : 0
total_collection_amount : 0
total_current_balance : 0
total_revolving_credit_limit : 0
loan_status : 28913


In [52]:
df.drop('loan_status', axis=1, inplace=True)

In [53]:
#Convert types like (float to integer, string to integers) to reduce the size and there isn't any integer stored as string to convert it

In [54]:
# Identify float and integer and object columns
float_cols = df.select_dtypes(include=['float']).columns
int_cols = df.select_dtypes(include=['integer']).columns

# Downcast floats to the smallest possible float type (e.g., float32)
df[float_cols] = df[float_cols].apply(pd.to_numeric, downcast='float')

# Downcast integers to the smallest possible integer type (e.g., int8, int16)
df[int_cols] = df[int_cols].apply(pd.to_numeric, downcast='integer')

In [55]:
# Use df.info to identify which columns has a lot of missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28913 entries, 0 to 28912
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   loan_id                       28913 non-null  int32  
 1   loan_amount                   28913 non-null  int32  
 2   funded_amount                 28913 non-null  int32  
 3   funded_amount_investor        28913 non-null  float64
 4   term                          28913 non-null  int8   
 5   batch_enrolled                28913 non-null  object 
 6   interest_rate                 28913 non-null  float32
 7   grade                         28913 non-null  object 
 8   sub_grade                     28913 non-null  object 
 9   home_ownership                28913 non-null  object 
 10  annual_income                 28913 non-null  float64
 11  verification_status           28913 non-null  object 
 12  payment_plan                  28913 non-null  object 
 13  l


 **# Convert Strings to the category Type, converting it to category can reduce memory usage by up to 90%.**

**Good for Category:** Grade, Sub Grade, Employment Duration, Home Ownership, Verification Status, Payment Plan, Initial List Status, Application Type. (These have a limited set of repeated values).

**Bad for Category:** Loan Title or Batch Enrolled (if they have thousands of unique, non-repeating names).

In [56]:
# Specific columns that are clearly categorical
cat_features = [
    'grade', 'sub_grade', 'home_ownership',
    'verification_status', 'payment_plan', 'initial_list_status', 'application_type'
]

for col in cat_features:
    df[col] = df[col].astype('category')


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28913 entries, 0 to 28912
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   loan_id                       28913 non-null  int32   
 1   loan_amount                   28913 non-null  int32   
 2   funded_amount                 28913 non-null  int32   
 3   funded_amount_investor        28913 non-null  float64 
 4   term                          28913 non-null  int8    
 5   batch_enrolled                28913 non-null  object  
 6   interest_rate                 28913 non-null  float32 
 7   grade                         28913 non-null  category
 8   sub_grade                     28913 non-null  category
 9   home_ownership                28913 non-null  category
 10  annual_income                 28913 non-null  float64 
 11  verification_status           28913 non-null  category
 12  payment_plan                  28913 non-null  

In [58]:
df.head()

,loan_id,loan_amount,funded_amount,funded_amount_investor,term,batch_enrolled,interest_rate,grade,sub_grade,home_ownership,...,total_received_late_fee,recoveries,collection_recovery_fee,med_collections_12m,application_type,last_week_pay,accounts_delinquent,total_collection_amount,total_current_balance,total_revolving_credit_limit
0,56492997,17120,10365,16025.082690,59,BAT2575549,12.163926,A,D1,RENT,...,0.088031,8.425776,0.731797,0,INDIVIDUAL,135,0,24,475442,4364
1,22540813,7133,11650,12615.795600,59,BAT2833642,6.564296,B,E3,MORTGAGE,...,0.041237,6.157008,0.992918,0,INDIVIDUAL,56,0,1,72412,2573
2,9862181,25291,25825,11621.280830,59,BAT1761981,14.729900,A,C3,MORTGAGE,...,0.021745,5.705077,0.281580,0,INDIVIDUAL,3,0,26,284825,19676
3,10097822,30781,9664,15375.823510,59,BAT5341619,10.523767,A,A2,RENT,...,0.092398,2.469688,0.959162,0,INDIVIDUAL,21,0,32,40842,7226
4,47771809,8878,9419,7176.647582,58,BAT4694572,9.997013,C,B3,OWN,...,0.010354,2.127835,0.402315,0,INDIVIDUAL,104,0,33,90825,26145


In [59]:
# Compare memory usage before and after
print(df.info(memory_usage='deep'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28913 entries, 0 to 28912
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   loan_id                       28913 non-null  int32   
 1   loan_amount                   28913 non-null  int32   
 2   funded_amount                 28913 non-null  int32   
 3   funded_amount_investor        28913 non-null  float64 
 4   term                          28913 non-null  int8    
 5   batch_enrolled                28913 non-null  object  
 6   interest_rate                 28913 non-null  float32 
 7   grade                         28913 non-null  category
 8   sub_grade                     28913 non-null  category
 9   home_ownership                28913 non-null  category
 10  annual_income                 28913 non-null  float64 
 11  verification_status           28913 non-null  category
 12  payment_plan                  28913 non-null  